In [26]:
from glob import glob
import os

def ncbi_id(db, db_id_query):
    from Bio import Entrez
    Entrez.email = "ad3002@gmail.com"
    handle = Entrez.esearch(db=db, retmax=100, term=db_id_query)
    query_id = Entrez.read(handle)["IdList"]
    return query_id

def ncbi_id_summary(db, db_id):
    from Bio import Entrez
    Entrez.email = "ad3002@gmail.com"
    handle = Entrez.esummary(db=db, id=db_id)
    summary = Entrez.read(handle)
    handle.close()
    return summary

def protid_to_organism(gff, protein_name):
    with open(gff) as fh:
        for line in fh:
            if protein_name in line:
                nid = ncbi_id("nucleotide", line.split("\t")[0])
                organism = ncbi_id_summary("nucleotide", nid)[0]["Title"]
                pid = line.split("\t")[8].split("Name=")[-1].split(";")[0]
                return [pid, organism]
                
def fasta_read(path_to_fasta_file):
    "Returns dictionary header:sequence"
    fasta = {}
    header = None
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    fasta[header] = "".join(seq)
                header = line[1:].split()[0]
                seq = []
            else:
                seq.append(line)
        if header:
            fasta[header] = "".join(seq)
    return fasta

In [32]:
sco_to_analize = ['30S ribosomal protein S7',
                  'molecular chaperone DnaK',
                  'F0F1 ATP synthase subunit alpha',
                  '50S ribosomal protein L6',
                  'aminoacyl-tRNA hydrolase',
                  'nucleotide exchange factor GrpE',
                  'uracil phosphoribosyltransferase',
                  'translation initiation factor IF-1',
                  '30S ribosome-binding factor RbfA',
                  'preprotein translocase subunit SecG',
                  'amidotransferase subunit GatB',
                  'cell wall cluster transcriptional repressor MraZ',
                  'deoxyribose-phosphate aldolase',
                  'methyltransferase RsmH',
                  '50S ribosomal protein L32']

## sco num_bact n_missed uniq_prots prots_in_multifasta multifasta_file
summary_table = []

anno_files = glob("/home/dzilov/data_zilov/data/mycoplasma/*/*/*.gff")

print(len(anno_files), 'genomes to analyze')


# read all cds fastas
all_fastas = []
for i in glob("/home/dzilov/data_zilov/data/mycoplasma/*/*/*cds_from_genomic.fna"):
    fasta = fasta_read(i)
    all_fastas.append(fasta)
    
# build multifasta for each sco by all bacteria

for sco in sco_to_analize:
    print(f"take {sco} to build multifasta")
    
    sco_joined = "_".join(sco.split())
    multifasta_file = f"/home/dzilov/data_zilov/data/mycoplasma/clusters/{sco_joined}.fasta"
    
    if os.path.exists(multifasta_file) and os.path.getsize(multifasta_file) != 0:
        print(f"Multifasta for {sco} already written, continue...")
        continue
    
    
    pids2bact = {} 
    n_missed = 0
    for file in anno_files:
        prot2bact = protid_to_organism(file, sco)
        if prot2bact:
            pids2bact[prot2bact[0]] = prot2bact[1]
        else:
            n_missed += 1
            print(f"MISSED {sco} in {file}")
    
    print(f'{len(pids2bact)} uniq prots in {sco}')
        
    multifasta = {}
    for pid, organism in pids2bact.items():
        for prots in all_fastas:
            for header, seq in prots.items():
                if pid in header:
                    short_organism = "_".join(organism.split(",")[0].split())
                    multifasta[f"{pid}_{short_organism}"]=seq
    
    

    with open(multifasta_file, "w") as fw:
        for h, s in multifasta.items():
            fw.write(f">{h}\n{s}\n")
            
    print(len(multifasta.keys()), f'prots added to multifasta {multifasta_file}')
    
    sco_summary = [sco, len(all_fastas), n_missed, len(pids2bact), len(multifasta.keys()), multifasta_file]

275 genomes to analyze
take 30S ribosomal protein S7 to build multifasta
Multifasta for 30S ribosomal protein S7 already written, continue...
take molecular chaperone DnaK to build multifasta
Multifasta for molecular chaperone DnaK already written, continue...
take F0F1 ATP synthase subunit alpha to build multifasta
Multifasta for F0F1 ATP synthase subunit alpha already written, continue...
take 50S ribosomal protein L6 to build multifasta
Multifasta for 50S ribosomal protein L6 already written, continue...
take aminoacyl-tRNA hydrolase to build multifasta
Multifasta for aminoacyl-tRNA hydrolase already written, continue...
take nucleotide exchange factor GrpE to build multifasta
Multifasta for nucleotide exchange factor GrpE already written, continue...
take uracil phosphoribosyltransferase to build multifasta
Multifasta for uracil phosphoribosyltransferase already written, continue...
take translation initiation factor IF-1 to build multifasta
Multifasta for translation initiation fa

MISSED 50S ribosomal protein L32 in /home/dzilov/data_zilov/data/mycoplasma/all_complete_genomes/GCF_000200735.1_ASM20073v1/GCF_000200735.1_ASM20073v1_genomic.gff
MISSED 50S ribosomal protein L32 in /home/dzilov/data_zilov/data/mycoplasma/all_complete_genomes/GCF_000238995.1_ASM23899v1/GCF_000238995.1_ASM23899v1_genomic.gff
68 uniq prots in 50S ribosomal protein L32
68 prots added to multifasta /home/dzilov/data_zilov/data/mycoplasma/clusters/50S_ribosomal_protein_L32.fasta


In [33]:
## MSA with MUSCLE for each protein cluster

#write commands for xargs

muscle_xargs = '/home/dzilov/data_zilov/data/mycoplasma/clusters/xargs_muscle.txt'
muscle_dir = '/home/dzilov/data_zilov/data/mycoplasma/clusters/muscle'

multifasta_list = glob('/home/dzilov/data_zilov/data/mycoplasma/clusters/*.fasta')


with open(muscle_xargs, 'w') as fw:
    for i in multifasta_list:
        out = os.path.join(muscle_dir, f"alignment_{os.path.basename(i)}")
        fw.write(f'muscle -in {i} -out {out}\n')

In [35]:
## phylogeny with IQTree for each protein cluster alignment

#write commands for xargs

iqtree_xargs = '/home/dzilov/data_zilov/data/mycoplasma/clusters/xargs_iqtree.txt'
iqtree_dir = '/home/dzilov/data_zilov/data/mycoplasma/clusters/iqtree'

alignment_list = glob('/home/dzilov/data_zilov/data/mycoplasma/clusters/muscle/*.fasta')

with open(iqtree_xargs, 'w') as fw:
    for i in alignment_list:
        out = os.path.join(iqtree_dir, os.path.splitext(os.path.basename(i))[0])
        fw.write(f'iqtree -T 1 --mem 100G -s {i} --prefix {out}\n')